# Lab 1: Planar Transformations, Homography Estimation & Image Mosaics

Welcome to your first hands-on lab! In this exercise, we'll dive into the fascinating world of image transformations and learn how to create stunning panoramas from multiple photos. Here's what you'll be doing:

### What You'll Learn:
1. **Projective Geometry and Homogeneous Coordinates:**  
   We'll start by exploring how different transformations affect an image. You'll apply various transformations and visually experience how each one changes the image. Afterward, we'll tackle image rectification, which is how we can "fix" or align images using transformations, just like we did in Seminar 1.

2. **Creating Panoramic Images:**  
   Using the concept of homographies, you'll stitch together images captured from slightly different angles to form a seamless panoramic image. Think of it as turning a series of photos into one wide, beautiful view!

### Deliverables:
1. 💻 **Complete the Code:**  
   You'll be given some starter code that you'll need to complete. Once you’ve filled in the missing parts, you'll submit your fully-executed Jupyter Notebook (.ipynb) with the code working as expected.

2. 🎥 **Explain Your Work:**  
   Record a video of yourself explaining the solutions to the questions in the lab, following the guidelines provided. It’s a great way to reinforce your understanding and demonstrate your progress!

Let's get started and happy coding! 🚀


<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color:rgba(255, 255, 255, 0);">
  <strong>🎥 Introduction</strong>:
  <ul>
    <li>Provide an overall explanation of the lab. Describe the goals of the lab clearly.</li>
    <li>State the problem you were trying to solve or explore in this lab.</li>
  </ul>
</div>


## 0. Setting up the environment

### 0.1 Environment Creation


Before starting the lab exercises, you must set up the working environment with any of the following options:
- A) Local setup with a Conda environment. (Recommended)
- B) Remote setup in Google Colab.
- C) Local setup with Python (Fallback option).


#### Option A: Local setup with a Conda environment

First of all, we recommend working on a conda environment. If conda is not installed, install it from https://www.anaconda.com/download/ .
Then, run the following commands in the command line:

```
conda create --name 3d_vision python=3.11 -y
conda activate 3d_vision
```
If you want, you can find further information about conda environments [here](https://medium.com/@viraj1604/comprehensive-guide-conda-virtual-environment-d70fafa7cf48).

Then, install the dependencies using `pip`:
```
pip install jupyter ipykernel pillow numpy scipy matplotlib plotly opencv-python pandas
python -m ipykernel install --user --name=3d_vision --display-name "Python (3d_vision)"
```
And make sure that the notebook is running with the recently created `3d_vision` environment.

In [ ]:
# import os
# env_name = os.environ['CONDA_DEFAULT_ENV'].split('/')[-1]
# print(f"Environment name: {env_name}")
# assert env_name == '3d_vision' # validating if you are running on 3d_vision conda environment

#### Option B: Remote setup in Google Colab

1. Open [Google Colab](https://colab.research.google.com) and upload this `.ipynb` file.
2. Zip the folder with all the material for the current lab session.
3. Upload the zipped folder anywhere in your Google Drive, share it for anyone with the link. You shoul obtain a link similar to `https://drive.google.com/file/d/14f1EP6UHr1Xk00X5n5Gjk8chBvCCX39/view?usp=share_link`.
4. Copy the part the id from the link, i.e. the alphanumerical id between the last two slashes. In the previous example it would be `14f1EP6UHr1Xk00X5n5Gjk8chBvCCX39`
5. Complete the cell below with the id and run it

In [ ]:
# id = # Add your link id here
# !mkdir Lab1
# !gdown {id}
# !unzip -q provided_files.zip -d Lab1
# %cd Lab1

In [ ]:
# You may need to change directory again to be able to import utils
# %cd provided_files

Now that we have uploaded the directories and files required to run this Notebook, let's install the dependencies to be able to import the required libraries. Run the cell below to install all libraries. 

In [ ]:
# %pip install pillow numpy scipy matplotlib plotly opencv-python pandas

#### Option C: Local setup with Python (Fallback option)

If you encountered any issues with the previous setups, you can install the necessary libraries directly into your Python environment using the following command:
```
pip install jupyter ipykernel pillow numpy scipy matplotlib plotly opencv-python pandas
```

### 0.2 Loading required libraries

In [ ]:
import cv2
import math
import random

import contextlib
import numpy as np
import plotly.express as px
import scipy.io as sio
from matplotlib import pyplot as plt
from matplotlib.widgets import Slider
from PIL import Image, ImageDraw
from scipy.ndimage import map_coordinates
import ipywidgets as widgets
from ipywidgets import VBox
from ipywidgets import interactive

from IPython.display import Markdown, clear_output
# import ipywidgets as widgets
# from ipywidgets import interactive
# from IPython.display import display, Markdown, clear_output


from utils import apply_H_fixed_image_size, line_draw, DLT_homography

## 1. Planar transformations

### 1.1 Applying a homography to an image

Below we provide the function `apply_H` which takes as input an image and a
transformation (specified in a $3\times 3$ matrix `H`) and applies the
desired transformation to the input image. 

**Q1.1** Examine and understand the code. Complete the code following 
the directions in the function.

In [ ]:
def get_transformed_pixels_coords(I, H, shift=None):
    """Transforms pixel coordinates using a homography matrix.
    
    Args:
        I (numpy.ndarray): Input image.
        H (numpy.ndarray): Homography matrix.
        shift (tuple, optional): Shift values for x and y coordinates. Defaults to None.
    
    Returns:
        numpy.ndarray: Transformed pixel coordinates.
    """
    # Get the height and width of the input image
    I_height, I_width = I.shape[:2]
    
    # Generate indices for all pixel positions in the input image.
    ys, xs = np.indices((I_height, I_width)).astype("float64")

    # Apply shift if provided
    if shift is not None:
        ys += shift[1]
        xs += shift[0]
        
    # Transform from cartesian to projective coordinates: (x, y) -> (x, y, 1)
    ws = np.ones((I_height, I_width))
    coords = np.stack((xs, ys, ws), axis=2)
    
    # Apply the homography transformation to the coordinates: H * (x, y, 1)^T = (x', y', w')^T
    coords_H = (H @ coords.reshape(-1, 3).T).T.reshape((I_height, I_width, 3))
    
    # Transform projective coordinates to cartesian: (x', y', w') -> (x'/w', y'/w')
    cart_H = coords_H[:, :, :2] / coords_H[:, :, 2:]
        
    return cart_H

def apply_H(I, H):
    """Apply homography transformation to an image.
    
    Args:
        I (numpy.ndarray): Input image.
        H (numpy.ndarray): Homography matrix.
    
    Returns:
        numpy.ndarray: Transformed image.
    """
    h, w = I.shape[:2]
    
    # corners
    c1 = np.array([1, 1, 1])
    c2 = np.array([w, 1, 1])
    c3 = np.array([1, h, 1])
    c4 = np.array([w, h, 1])
    
    # Compute the transformed homogeneous image corners according to H.
    # Normalize the homogeneous coordinates so as the third coordinate is always 1.
    # Call the transformed corners: Hc1, Hc2, Hc3, Hc4

    # TODO: Transform corners according to H
    # NOTE: * stands for element-wise multiplication while @ is typical mat/vec multiplication
    Hc1 = H @ c1 # transformed corner c1
    Hc2 = H @ c2 # transformed corner c2
    Hc3 = H @ c3 # transformed corner c3
    Hc4 = H @ c4 # transformed corner c4
    
    # TODO: Normalize homogeneous coordinates
    # NOTE: When using /= the following error appears
    # UFuncTypeError: Cannot cast ufunc 'divide' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'
    # so I will just stick to more standard syntax
    Hc1 = Hc1 / Hc1[2]
    Hc2 = Hc2 / Hc2[2]
    Hc3 = Hc3 / Hc3[2]
    Hc4 = Hc4 / Hc4[2]
    
    # Compute extremal transformed corner coordinates
    xmin = np.round(np.min([Hc1[0], Hc2[0], Hc3[0], Hc4[0]]))
    xmax = np.round(np.max([Hc1[0], Hc2[0], Hc3[0], Hc4[0]]))
    ymin = np.round(np.min([Hc1[1], Hc2[1], Hc3[1], Hc4[1]]))
    ymax = np.round(np.max([Hc1[1], Hc2[1], Hc3[1], Hc4[1]]))
    
    # Calculate size of output image
    size_x = math.ceil(xmax - xmin + 1)
    size_y = math.ceil(ymax - ymin + 1)

    # Create an empty output image
    out = np.zeros((size_y, size_x, 3))

    # Invert the homography matrix
    H_inv = np.linalg.inv(H)
    
    # Define shift for map_coordinates function
    shift = (xmin, ymin)
    
    # Get coordinates for interpolation
    interpolation_coords = get_transformed_pixels_coords(out, H_inv, shift=shift)
    interpolation_coords[:, :, [0, 1]] = interpolation_coords[:, :, [1, 0]]
    interpolation_coords = np.swapaxes(np.swapaxes(interpolation_coords, 0, 2), 1, 2)

    # Apply interpolation to each channel of the input image (R, G, and B)
    out[:, :, 0] = map_coordinates(I[:, :, 0], interpolation_coords)
    out[:, :, 1] = map_coordinates(I[:, :, 1], interpolation_coords)
    out[:, :, 2] = map_coordinates(I[:, :, 2], interpolation_coords)
  
    return out.astype("uint8")

Now we are going to test the completed function `apply_H.m`
with a hierarchy of 2D transformations. 


**Q1.2.** We want to apply a rotation of $30^o$ to the image. Which is the appropriate
expression of matrix `H` below?

In [ ]:
# TODO: Write the expression for H
rot = np.deg2rad(30)
H = np.array([
    [np.cos(rot), -np.sin(rot), 0],
    [np.sin(rot),  np.cos(rot), 0],
    [0,            0,           1]
])

# Convert matrix H to a LaTeX formatted string with actual values
latex_str = r"$H = \begin{bmatrix} " + \
    f"{H[0, 0]:.4f} & {H[0, 1]:.4f} & {H[0, 2]:.4f} \\\\" + \
    f" {H[1, 0]:.4f} & {H[1, 1]:.4f} & {H[1, 2]:.4f} \\\\" + \
    f" {H[2, 0]:.4f} & {H[2, 1]:.4f} & {H[2, 2]:.4f}" + \
    r"\end{bmatrix}$"

# Display it as Markdown
display(Markdown(latex_str))

img_path = "./Data/mondrian.jpg"
I = Image.open(img_path)
It = apply_H(np.array(I), H)

plt.subplot(1,2,1)
plt.title("Original image")
plt.imshow(I)
plt.subplot(1,2,2)
plt.title("Image rotated 30°")
plt.imshow(It)
plt.show()

<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color:rgba(255, 255, 255, 0);">
  <ul style="margin: 0; padding-left: 20px;">
    <strong>🎥 Video Question:</strong> Regarding H, which kind of transformation is it? Why?
  </ul>
</div>

### 1.2 Decomposing of a transformation

Now apply the following transformation to the Mondrian image (complete the cell code below):
$$ H = \left(\begin{array}{ccc}
    2 & -1 & 1 \\
    1 & 1 & 3 \\
    0 & 0 & 1 \\
   \end{array} \right)
$$

In [ ]:
# TODO: Write the matrix H
H = np.array([
    [2, -1, 1],
    [1,  1, 3],
    [0,  0, 1]
])

It = apply_H(np.array(I), H)

plt.figure(figsize=(10,3))
plt.subplot(1, 2, 1)
plt.title("Original image")
plt.imshow(I)
plt.subplot(1, 2, 2)
plt.title("Transformed image")
plt.imshow(It)
plt.show()

<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color:rgba(255, 255, 255, 0);">
  <strong>🎥 Video Question:</strong> Regarding this new homography, which kind of transformation is it? Why?
</div>

💻 **Code question**: Use the SVD decomposition (function `np.linalg.svd`) to express the
transformation $H$ as a composition of two rotations, a pure anisotropic
scaling and a pure translation (the rotations may also include a reflection).

In [ ]:
# TODO: Decompose H
# construct T which only has translation info
T = np.identity(3)
T[0, 2] = H[0, 2]
T[1, 2] = H[1, 2]

# get the rotation and translation matrix A_p (A prime) from T and H
A_p = np.linalg.inv(T) @ H

# decompose A_p
# first rotation is in U
# scaling factors are the singular values
# second rotation is Vh
U, S, Vh = np.linalg.svd(A_p)

# so that it is a 3x3 matrix
S = np.diag(S)

display(U)

'''
decompose s.t. first rotation, scaling, second rotation, translation

first (T translation mat) is    I T
            0 1

second (rot + scaling mat) is   A 0
            0 1
svd the second

first @ second = H
'''

<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color:rgba(255, 255, 255, 0);">
  <strong>🎥 Video Question:</strong><br>
  <li> Which are the matrices corresponding to each one of the transformations?<br>
  <li> Which is the appropriate order of composition?
</div>


To transform the image, we will apply the matrix transformations from right to left because:
$$I'=HI=TU'S'V'^HI$$

Apply these matrices to the image in the proper order, one by one,  with the function `apply_H` and verify that you get the same result than applying all the transformations at the same time using a single matrix. Provide in the cell code below all the commands used for this verification.

In [ ]:
# Visualise the transformations step by step

# TODO: Apply transformations step by step
# NOTE: at first i computed it first with U for I1 and Vh for I3
# but it is computed from right to left
# e.g.: for each point i xi_prime = H @ x_i = T @ U @ S @ Vh @ x_i
I1 = apply_H(np.array(I), Vh)
I2 = apply_H(np.array(I1), S)
I3 = apply_H(np.array(I2), U)
I4 = apply_H(np.array(I3), T)

# TODO: Add your list of transformed images step by step
images = [I, I1, I2, I3, I4]

plt.figure(figsize=(20,4))
titles = ["Original image", "First rotation", "Scaling", "Second rotation", "Translation (not visible)"]
for i, (image, title) in enumerate(zip(images, titles), start=1):
    plt.subplot(1, 5, i)
    plt.title(title)
    plt.imshow(image)
plt.show()

<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color:rgba(255, 255, 255, 0);">
  <strong>🎥 Video Question:</strong> Do the sequence of transformations look right to you? Why?<br>
  <em>(Don't worry if there is a translation difference in the last image)</em>
</div>

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1)
plt.title("Image transformed in a single step")
plt.imshow(It)
plt.subplot(1, 2, 2)
plt.title("Image transformed step by step")
# TODO: Add the final image after all transformations
plt.imshow(I4)
plt.show()

<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color:rgba(255, 255, 255, 0);">
  <strong>🎥 Video Question</strong>: Do the original and transformed image look similar?<em>(Don't worry if there is a translation difference)</em></li>
</div>

## 2. Image Rectification

We are going to test two image rectification methods
1. Direct Linear Transformation
2. Stratified Method

### 2.1 Direct Linear Transformation

#### 2.1.1 Interactive method

In the first one we are going to remove the projective distortion from a perspective image of a plane (flat surface in the 3D world). The key idea is to select four coplanar points that should be mapped to a rectangle.
 

In [ ]:
img_path = "./Data/metro2.png"
I = Image.open(img_path)
plt.imshow(I)
plt.title("Original image")
plt.show()

In [ ]:
p1 = np.array([48.0, 537.0, 1])
p2 = np.array([344.0, 480.0, 1])
p3 = np.array([341.0, 329.0, 1])
p4 = np.array([75.0, 265.0, 1])

q1 = np.array([10.0, 200.0, 1])
q2 = np.array([300.0, 200.0, 1])
q3 = np.array([300.0, 10.0, 1])
q4 = np.array([10.0, 10.0, 1])

X = np.array([p1, p2, p3, p4])
X = X.T
Y = np.array([q1, q2, q3, q4])
Y = Y.T

I = Image.open(img_path)
canv2 = ImageDraw.Draw(I)
for i in range(4):
    canv2.ellipse((X[0,i], X[1,i], X[0,i]+7, X[1,i]+7), fill = 'cyan', outline ='cyan')
    

plt.figure(figsize=(18.5, 10.5))
plt.title("Original image with selected points to become a rectangle")
plt.imshow(I)
plt.show()

The purpose now is to find the homography `H` that relates the two sets of
points, encoded in the matrices `X` and `Y`, by using the normalised Direct Linear Transformation (DLT) algorithm.
This algorithm is implemented in the provided function `DLT_homography` in the utils file. The
matrix `H` is obtained by calling this function with the two matrices `X` and `Y` of
homogeneous coordinates as inputs. Examine the function `DLT_homography` and identify the different steps of the algorithm studied in class.

In [ ]:
# TODO: Apply the DLT homography
H = DLT_homography(X, Y)
It = apply_H(np.array(I), H)

print(H)

plt.figure(figsize=(18.5, 10.5))
plt.title("Image transformed with DLT")
plt.imshow(It)
plt.show()

<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color:rgba(255, 255, 255, 0);">
  <strong>🎥 Video Question</strong>: How does the DLT algorithm work in this example? Explain it briefly. <br>
  <em>(You can show it in the DLT_homography function)</em>
</div>

#### 2.1.2 Rectifying a picture of a paper **(OPTIONAL)**

Now, we are going rectify an image of a paper. Our goal is to apply a transformation to visualize the paper in a flattened position, as if we were viewing the original PDF file.

In [ ]:
img_path = "./Data/paper_picture_low_resolution.jpg"
I = Image.open(img_path)
fig = px.imshow(I)
fig.show()

Select the corners in the image above. For an accurate selection, you can zoom in by drawing a rectangle in the image, and then press autoscale from the options at the top right.

In the cell below, what should be the values for `p1_paper`, `p2_paper`, `p3_paper`, and `p4_paper`? Fill them in, knowing that `pi_paper` will be transformed to the position `qi` for *i* ranging from 1 to 4.

In [ ]:
# TODO: Find the coordinates of the points below
# Notice the order of axes in numpy, 1st horizontal and 2nd vertical.
# Being the origin (0,0) at the top-left of the image.
# NOTE: order bugs me out
p1_paper = np.array([1460, 803, 1])
p2_paper = np.array([1316, 288, 1])
p3_paper = np.array([403, 132, 1])
p4_paper = np.array([27, 503, 1])

q1 = np.array([0, 2830, 1]) # lower left corner
q2 = np.array([2000, 2830, 1]) # lower right corner
q3 = np.array([2000, 0, 1]) # upper right corner
q4 = np.array([0, 0, 1]) # upper left corner

X_paper = np.array([p1_paper, p2_paper, p3_paper, p4_paper])
X_paper = X_paper.T
Y = np.array([q1, q2, q3, q4])
Y = Y.T

# Displaying the image with chosen points
plt.figure(figsize=(18, 10))
plt.title("Picture of paper with selected corners in red")
for i, (x, y) in enumerate(zip(X_paper[0], X_paper[1]), start=1):
    plt.scatter(x=x, y=y, c='r', s=20)
    plt.text(x, y, f'$p_{i}$', fontsize=15, ha='right', va='bottom', color='red', weight='bold')
plt.imshow(I)
plt.show()

In the cell below, compute H using the DLT algorithm, and print the values of H and explain what type of transformation it represents.

In [ ]:
# TODO: Apply the DLT to find the homography
H = DLT_homography(X_paper, Y)

print(H)
corners = [-1000, 6000, -2000, 5000]
It = apply_H_fixed_image_size(np.array(I), H, corners)

# Display the transformed picture
plt.figure(figsize=(18, 10))
plt.title("Transformed picture of the paper through DLT")
plt.imshow(It)
for i, (x, y) in enumerate(zip(Y[0]-corners[0], Y[1]-corners[2]), start=1):
    plt.scatter(x=x, y=y, c='r', s=20)
    plt.text(x, y, f'$q_{i}$', fontsize=15, ha='right', va='bottom', color='red', weight='bold', style='oblique') 
plt.show()

Let's crop the image to isolate the contents of the paper, excluding any surrounding background or unwanted elements.

In [ ]:
# Calculate the minimum and maximum values for X and Y
xmin, xmax = np.min(Y[0]) - corners[0], np.max(Y[0]) - corners[0]
ymin, ymax = np.min(Y[1]) - corners[2], np.max(Y[1]) - corners[2]

# Crop the the rectified image (notice the order of y and x)
It_cropped = It[ymin:ymax, xmin:xmax]

# Display the rectified portion of the image It
plt.figure(figsize=(8, 14))
plt.title("Rectified picture of the paper through DLT")
plt.imshow(It_cropped)
plt.axis('off')
plt.show()

### 2.2 Stratified method

In this part we will test a method for image rectification which works in two steps:
1. Affine rectification (As in Seminar 1)
2. Metric rectification (Optional).

#### 2.2.1-A) Affine rectification

This first step uses the line at infinity. The rectification is based on transforming the identified 
image of the line at infinity to its canonical position of $\ell_{\infty} = (0, 0, 1)^T$. 
This is done by a projective transformation built from the coordinates of the imaged 
line at infinity, $\ell = (\ell_1, \ell_2, \ell_3)^T$, provided that $\ell_3 \neq 0$. 
The idea 
is illustrated in the following figure:

![Title](./Data/Lab1-Stratified_method.drawio.png)

<!-- ![Title](https://www.robots.ox.ac.uk/~vgg/hzbook/hzbook2/WebPage/pngfiles/projgeomfigs-affine_rect.png) -->

We are going to compute $\ell$, the imaged $\ell_{\infty}$, as the intersection 
of the two pair of lines formed by the four selected points in the previous exercise.
For that, we are going to use the dual properties of homogeneous points and lines that define the line joining two points and the intersection of lines.

The first approach to compute the line at infinity will be to use the four points manually selected previously. 

**Q2.1**
Compute the vector `l1` representing the line that joins the two upper selected points.
Compute the vector `l2` representing the line that joins the two lower selected points.
Compute the vector `l3` representing the line that joins the two left selected points.
Compute the vector `l4` representing the line that joins the two right selected points.

SUGGESTION: You may use the numpy function `cross`.


In [ ]:
img_path = "./Data/metro2.png"
I = Image.open(img_path)

# Display image with points
plt.title("Original image with selected points")
plt.imshow(I)
for i, (x, y) in enumerate(zip(X[0], X[1]), start=1):
    plt.scatter(x=x, y=y, c='c', s=5)
    plt.text(x, y, f'p{i}', fontsize=8, ha='right', va='bottom', color='cyan')
plt.show()

In [ ]:
# TODO: Compute the line vectors
# NOTE: Should the order matter ???
# NOTE: X is organized column-wise apparently
l1 = np.cross(X[:, 3], X[:, 2])
l2 = np.cross(X[:, 0], X[:, 1])
l3 = np.cross(X[:, 0], X[:, 3])
l4 = np.cross(X[:, 1], X[:, 2])

You may visualize the computed lines to check that your result is correct. 
Use the following commands to visualize the lines.

In [ ]:
# Function required to display lines
def get_line_points(l, I):
    x = np.arange(0, np.array(I).shape[1])
    y = (-l[0] * x - l[2]) / l[1]
    return x, y

# Display image with lines and points
plt.figure(figsize=(18.5, 10.5))
plt.title("Original image with computed lines and points")
for i, l in enumerate([l1, l2, l3, l4], start=1):
    plt.plot(*get_line_points(l, I), label=f'$l_{i}$')
for i, (x, y) in enumerate(zip(X[0], X[1]), start=1):
    plt.scatter(x=x, y=y, c='c', s=5)
    plt.text(x, y, f'$p_{i}$', fontsize=12, ha='right', va='bottom', color='cyan', weight='bold') 
plt.imshow(I)
plt.legend()
plt.show()

**Q2.2** 
Give the expression of a matrix `H_aff_rect` that maps the line 
$\ell$ to the line $\ell_{\infty} = (0, 0, 1)^T$. Complete the code below in order to rectify the image by applying the computed homography. Compute also the transformed lines in order to visualize them as well.

In [ ]:
# TODO: Compute the vanishing point vectors and the line that passes through them
v1 = np.cross(l1, l2)
v2 = np.cross(l3, l4)
l = np.cross(v1, v2)
l = l / l[2] # Normalize the 3rd coordinate

In [ ]:
# TODO: Define the homography that affinely rectifies the image
H_aff_rect = np.identity(3)
H_aff_rect[2, :] = l # set last row to l

print("H_aff_rect:", H_aff_rect, sep="\n")

# TODO: Compute the transformed image
corners = [0, 1000, 0, 1000]
I_aff_rect = apply_H_fixed_image_size(np.array(I), H_aff_rect, corners)

# TODO: Compute the transformed lines
H_aff_rect_mt = np.linalg.inv(H_aff_rect).T
lr1 = H_aff_rect_mt @ l1
lr2 = H_aff_rect_mt @ l2
lr3 = H_aff_rect_mt @ l3
lr4 = H_aff_rect_mt @ l4

# TODO: Compute the transformed points
q1 = H_aff_rect @ X[:, 0]
q2 = H_aff_rect @ X[:, 1]
q3 = H_aff_rect @ X[:, 2]
q4 = H_aff_rect @ X[:, 3]


# show the transformed lines and points in the transformed image
I_tr = Image.fromarray(I_aff_rect, 'RGB')
size = I_tr.size

plt.figure(figsize=(18.5, 10.5))
plt.title("Rectified image with rectified lines and points")

# Display rectified lines
for i, line in enumerate([lr1, lr2, lr3, lr4], start=1):
    plt.plot(*get_line_points(line, I_tr), label=f'$lr_{i}$')

# Display rectified points
for i, (x, y, z) in enumerate([q1, q2, q3, q4], start=1):
    x_norm, y_norm = x/z, y/z
    plt.scatter(x=x_norm, y=y_norm, c='c', s=15)
    plt.text(x_norm, y_norm, f'$q_{i}$', fontsize=12, ha='right', va='bottom', color='cyan', weight='bold') 

plt.legend()
plt.imshow(I_tr)
plt.show()

<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color:rgba(255, 255, 255, 0);">
  <strong>🎥 Video Questions</strong>:
  <ul style="margin-top: 5px; padding-left: 20px;">
    <li>What is the interpretation of the last row of the affine rectification matrix?</li>
    <li>Which is the geometric relation of each pair of transformed lines before and after the transformation? (i.e., l₁ with respect to l₂, lr₁ with respect to lr₂, and so on with 3 and 4.)</li>
    <li>What would be the result of the cross product of each pair of transformed lines (lr₁·lr₂, and lr₃·lr₄), and why?</li>
  </ul>
</div>


#### 2.2.1-B) Semi-automatic Affine Rectification **(OPTIONAL)**

Now, instead of manually selecting the projected parallel lines in the image we will use the vanishing points estimated by an automatic method for detecting vanishing points in urban scenes. It is based on the detection of line segments in the image and then identifying sets of converging segments and their intersection point by finding point alignments in a proper different space (more details, online demo and code available in the webage of the related publication http://www.ipol.im/pub/art/2017/148/).
The result of the method applied to our image of interest is provided in the folder called 'vanishing_points'. The algorithm has detected four different vanishing points: their coordinates are given in matrix `vps` read in the code below. The lines corresponding to each of these points are depicted in the provided images in the folder 'vanishing_points'.

Identify the proper pair of vanishing points to affinely rectify the image (justify your answer) and provide the code to obtain the rectified image with the new estimated vanishing points.

In [ ]:
mat_content = sio.loadmat('./Data/vanishing_points/vps.mat')
vps=mat_content['vps']
print(vps)

# complete the code that affinely rectifies the image and shows the transformed image ...
'''
Vanishing points 1 and 3 seem to fall inside the image which does not make sense
VP 2 seems to be the vanishing point between lines 1 and 2
VP 4 seems to be between lines 3 and 4
'''
l = np.cross(np.append(vps[:, 1], [1]), np.append(vps[:, 3], [1]))
l = l / l[2]

In [ ]:
# NOTE: this step does not change w.r.t. the last method

H_aff_rect = np.identity(3)
H_aff_rect[2, :] = l # set last row to l

print("H_aff_rect:", H_aff_rect, sep="\n")

corners = [0, 1000, 0, 1000]
I_aff_rect = apply_H_fixed_image_size(np.array(I), H_aff_rect, corners)

# NOTE: even though the points and lines are not used in this section,
# i think it is cool to see the same points and lines transformed to
# see if there is any significant change
H_aff_rect_mt = np.linalg.inv(H_aff_rect).T
lr1 = H_aff_rect_mt @ l1
lr2 = H_aff_rect_mt @ l2
lr3 = H_aff_rect_mt @ l3
lr4 = H_aff_rect_mt @ l4

q1 = H_aff_rect @ X[:, 0]
q2 = H_aff_rect @ X[:, 1]
q3 = H_aff_rect @ X[:, 2]
q4 = H_aff_rect @ X[:, 3]

# show the transformed lines and points in the transformed image
I_tr = Image.fromarray(I_aff_rect, 'RGB')
size = I_tr.size

plt.figure(figsize=(18.5, 10.5))
plt.title("Rectified image with rectified lines and points (Semi-automatic)")

# Display rectified lines
for i, line in enumerate([lr1, lr2, lr3, lr4], start=1):
    plt.plot(*get_line_points(line, I_tr), label=f'$lr_{i}$')

# Display rectified points
for i, (x, y, z) in enumerate([q1, q2, q3, q4], start=1):
    x_norm, y_norm = x/z, y/z
    plt.scatter(x=x_norm, y=y_norm, c='c', s=15)
    plt.text(x_norm, y_norm, f'$q_{i}$', fontsize=12, ha='right', va='bottom', color='cyan', weight='bold') 

plt.legend()
plt.imshow(I_tr)
plt.show()

#### 2.2.2 Metric Rectification **(OPTIONAL)**

**Metric rectification** typically comes after **affine rectification**, which corrects for the more basic linear distortions in the image (such as translation, rotation, and scaling). While affine rectification brings the image closer to the correct perspective, it does not necessarily preserve true distances or angles. Metric rectification goes a step further by adjusting the image to match the actual geometric properties of the scene.

In other words, metric rectification allows us to adjust the image so that distances and angles in the image correspond to their true physical values, as if the image were taken from an orthographic or "perfect" perspective.

In this example we would like the

Play with interactive sliders to see how an affine transformation affects the rectified image. Can you find an affine transformation $H$ ($H_a^{-1}$ from the diagram above) that rotates and scales the image such that it appears as if the metro door is in front of the camera?

$$H? \quad \Rightarrow \left\{
\begin{aligned}
    l_{r1}, l_{r2} &\text{ become horizontal lines} \\
    l_{r3}, l_{r4} &\text{ become vertical lines} \\
    l_{r1}, l_{r2} &\perp l_{r3}, l_{r4}
\end{aligned}
\right.$$

In [ ]:
# --- Create the figure and axis ---
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
plt.close(fig) # Keep this closed, display happens inside 'out'

# --- Create an Output widget ---
out = widgets.Output()

# --- Initial affine homography matrix ---
H_a = np.eye(3)

# --- Function to apply transformation and update the Output widget ---
def update_output(a11, a12, a21, a22, tx, ty):
    global H_a
    H_a = np.array([[a11, a12, tx], [a21, a22, ty], [0, 0, 1]])

    rows, cols = I_aff_rect.shape[:2]
    transformed_image = cv2.warpPerspective(
        I_aff_rect, H_a, (cols, rows),
        flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0)
    )

    ax.clear()
    ax.imshow(transformed_image)
    ax.set_title(f"Affine Transformed Image")
    ax.axis('off')

    latex_str = r"$H_a = \begin{bmatrix} " + \
        f"{H_a[0,0]:.4f} & {H_a[0,1]:.4f} & {H_a[0,2]:.4f} \\\\" + \
        f" {H_a[1,0]:.4f} & {H_a[1,1]:.4f} & {H_a[1,2]:.4f} \\\\" + \
        f" {H_a[2,0]:.4f} & {H_a[2,1]:.4f} & {H_a[2,2]:.4f}" + \
        r"\end{bmatrix}$"

    # Update the Output widget's content
    with out:
        clear_output(wait=True)
        display("Don't worry if the image and the matrix show multiple times. It's an error form our side.")
        display(fig)
        display(Markdown(latex_str))

# --- Create the sliders individually ---
slider_opts = {'continuous_update': False, 'readout_format': '.2f'}
a11_slider = widgets.FloatSlider(value=1.0, min=-3.0, max=3.0, step=0.01, description='a11:', **slider_opts)
a12_slider = widgets.FloatSlider(value=0.0, min=-3.0, max=3.0, step=0.01, description='a12:', **slider_opts)
a21_slider = widgets.FloatSlider(value=0.0, min=-3.0, max=3.0, step=0.01, description='a21:', **slider_opts)
a22_slider = widgets.FloatSlider(value=1.0, min=-3.0, max=3.0, step=0.01, description='a22:', **slider_opts)
tx_slider = widgets.FloatSlider(value=0, min=-200, max=200, step=1, description='Translate X:', continuous_update=False, readout_format='.0f')
ty_slider = widgets.FloatSlider(value=0, min=-200, max=200, step=1, description='Translate Y:', continuous_update=False, readout_format='.0f')

# --- Use interactive_output to link sliders to the function ---
interactive_updater = widgets.interactive_output(update_output, {
    'a11': a11_slider, 'a12': a12_slider, 'a21': a21_slider,
    'a22': a22_slider, 'tx': tx_slider, 'ty': ty_slider
})

# --- Arrange the UI: Sliders and the Output widget ---
controls = VBox([a11_slider, a12_slider, a21_slider, a22_slider, tx_slider, ty_slider])
# The 'out' widget is where the function sends its display calls.
ui = VBox([controls, out])

# --- Display the UI ---
display(ui)

In [ ]:
H_a = np.array([
    [1.71, 0.23, 0],
    [-0.69, 1.24, 0],
    [0, 0, 1]
])

We know that $H = H_a * H_p$, so we can do the following by pre-computing H and transforming the original image, or we can concatenate the inverse transformation H_a to the previously done transformation (H_aff_rect). 

In [ ]:
print("H_a:", H_a, sep="\n")

corners = [0, 1500, 0, 800]
I_rect = apply_H_fixed_image_size(I_aff_rect, H_a, corners)

# NOTE: even though the points and lines are not used in this section,
# i think it is cool to see the same points and lines transformed to
# see if there is any significant change
H_a_mt = np.linalg.inv(H_a).T
lr1 = H_a_mt @ H_aff_rect_mt @ l1
lr2 = H_a_mt @ H_aff_rect_mt @ l2
lr3 = H_a_mt @ H_aff_rect_mt @ l3
lr4 = H_a_mt @ H_aff_rect_mt @ l4

q1 = H_a @ H_aff_rect @ X[:, 0]
q2 = H_a @ H_aff_rect @ X[:, 1]
q3 = H_a @ H_aff_rect @ X[:, 2]
q4 = H_a @ H_aff_rect @ X[:, 3]

# show the transformed lines and points in the transformed image
I_tr = Image.fromarray(I_rect, 'RGB')
size = I_tr.size

plt.figure(figsize=(18.5, 10.5))
plt.title("Rectified image with rectified lines and points")

# Display rectified lines
for i, line in enumerate([lr1, lr2, lr3, lr4], start=1):
    plt.plot(*get_line_points(line, I_tr), label=f'$lr_{i}$')

# Display rectified points
for i, (x, y, z) in enumerate([q1, q2, q3, q4], start=1):
    x_norm, y_norm = x/z, y/z
    plt.scatter(x=x_norm, y=y_norm, c='c', s=15)
    plt.text(x_norm, y_norm, f'$q_{i}$', fontsize=12, ha='right', va='bottom', color='cyan', weight='bold') 

plt.legend()
plt.imshow(I_tr)
plt.show()

## 3. Image mosaics

The images that form the mosaic are related by homographies that we will need to compute.  The process will have the following steps:

- The first step will be to compute keypoints in the images that we can use to find correspondences.  We will use SIFT or ORB keypoints and features.
- Next we will compute correspondences between sets of keypoint features. We will use existing code for these first two steps.
- From the correspondences we will compute the homography that maps one image into the another. Since some correspondences may be erroneous, the computation will have to be robust to outliers. We will use the "RANdom SAmple Consensus" (RANSAC) method that you will have to complete.
- Finally, with the homographies computed, we will map all the images into a common canvas by using a variant of the  `apply_H` function from the last assignment.



### 3.0 Visualize the original images

In [ ]:
# Reading images using OpenCV
img1c = cv2.imread('Data/llanes_a.jpg', cv2.IMREAD_COLOR)
img2c = cv2.imread('Data/llanes_b.jpg', cv2.IMREAD_COLOR)
img3c = cv2.imread('Data/llanes_c.jpg', cv2.IMREAD_COLOR)

# Converting color space from BGR to RGB using OpenCV
img1c = cv2.cvtColor(img1c, cv2.COLOR_BGR2RGB)
img2c = cv2.cvtColor(img2c, cv2.COLOR_BGR2RGB)
img3c = cv2.cvtColor(img3c, cv2.COLOR_BGR2RGB)

# Creating a Matplotlib figure for displaying images
plt.figure(figsize=(20,5))

# Iterating over the images and plotting them
for i, image in enumerate([img1c, img2c, img3c], start=1):
    plt.subplot(1, 3, i)
    plt.title(f"Image {i}")
    plt.imshow(image)

# Displaying the Matplotlib figure with images
plt.show()

### 3.1 Compute image correspondences

The first step is to read the images and to compute their keypoints. The images are RGB. We have to convert them to gray scale with values in order to compute the keypoints on them. Then, we compute the keypoints and descriptors of every image. For the parts where the content of the two images coincide, you can visually check that many of the detected points are detected in both images. We want to find these _correspondences_.

In [ ]:
# Reading images using OpenCV in gray scale
img1 = cv2.imread('Data/llanes_a.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('Data/llanes_b.jpg', cv2.IMREAD_GRAYSCALE)
img3 = cv2.imread('Data/llanes_c.jpg', cv2.IMREAD_GRAYSCALE)

To match the keypoints between two images, we need to assign to each keypoint in the first image the one that has the most similar descriptor in the second image. 

Execute the following code to find image correspondences using SIFT [1].

[1] David Lowe. Object recognition from local scale-invariant features. ICCV, 1150-1157, 1999.

<!-- [3] Herbert Bay, Tinne Tuytelaars, Luc Van Gool. Surf: Speeded up robust features. ECCV, 404-417, 2006. -->

In [ ]:
# Initiating SIFT detector
sift = cv2.SIFT_create(3000)

# Finding the keypoints and descriptors
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# Drawing only keypoints location, not size and orientation
img1b = cv2.drawKeypoints(img1, kp1, None, color=(0,255,0), flags=0)
img2b = cv2.drawKeypoints(img2, kp2, None, color=(0,255,0), flags=0)

# Showing keypoints in images
plt.figure(figsize=(13,5))
plt.suptitle("Keypoints extracted using SIFT")
for i, image in enumerate([img1b, img2b], start=1):
    plt.subplot(1, 2, i)
    plt.title(f"Keypoints for Image {i}")
    plt.imshow(image)
plt.show()

# Keypoint matching
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches_12 = bf.match(des1, des2)

# Show matches
img_12 = cv2.drawMatches(img1, kp1, img2, kp2, matches_12, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure(figsize=(18.5, 10.5))
plt.title("Matching keypoints from images 1 and 2 with SIFT")
plt.imshow(img_12)
plt.show()

**Q3.1**
Compute and visualize the keypoints and matchings between image 2 and 3. Write the commands you used for that.

In [ ]:
# TODO: Correspondences between image 2 and 3
kp3, des3 = sift.detectAndCompute(img3, None)

img2b = cv2.drawKeypoints(img2, kp2, None, color=(0,255,0), flags=0)
img3b = cv2.drawKeypoints(img3, kp3, None, color=(0,255,0), flags=0)

# Showing keypoints in images
plt.figure(figsize=(13,5))
plt.suptitle("Keypoints extracted using SIFT")
for i, image in enumerate([img1b, img2b], start=1):
    plt.subplot(1, 2, i)
    plt.title(f"Keypoints for Image {i}")
    plt.imshow(image)
plt.show()

# Keypoint matching
matches_23 = bf.match(des2, des3)

# Show matches
img_23 = cv2.drawMatches(img2, kp2, img3, kp3, matches_23, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure(figsize=(18.5, 10.5))
plt.title("Matching keypoints from images 1 and 2 with SIFT")
plt.imshow(img_23)
plt.show()

<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color:rgba(230, 242, 255, 0) 255, 255);">
  <strong>🎥 Video Questions</strong>:
  <ul style="margin-top: 5px; padding-left: 20px;">
    <li>Explain the result after applying keypoint matching.</li>
  </ul>
</div>


### 3.2 Compute the homography (robust DLT algorithm) between image pairs

We want now to compute the homography that relates each pair of images. From  the last assignment, we have a function called `DLT_homography` that computes a homography given a set of correspondences. Unfortunately, this only works when all of the correspondences are correct, which is not the case in most practical applictions as the current one. This time, we will need to use the RANSAC method in order to find the correct correspondences and discard the others.

For that we use the functions `Ransac_DLT_homography` and `find_homography_inliers` that you have 
to complete below. Complete the `Ransac_DLT_homography` function by answering the following questions:

**Q3.2** Set the number of samples to choose randomly and that will define the model to test in each trial. (second input parameter of function `random.sample`).

**Q3.3** Complete the function `Inliers` by computing the geometric error of the correspondences given the homography.

In [ ]:
def find_homography_inliers(H, points1, points2, th):
    """Finds the inliers between two sets of points using a homography matrix.
    
    Args:
        H (numpy.ndarray): Homography matrix.
        points1 (numpy.ndarray): Points from the first image.
        points2 (numpy.ndarray): Points from the second image.
        th (float): Threshold for considering a point as an inlier.
    
    Returns:
        numpy.ndarray: Indices of the inliers in the consensus set.
    """
    try:
        H_inv = np.linalg.inv(H)
    except np.linalg.LinAlgError:
        print("Matrix H is not invertible.")
        return np.empty(0)  # Return empty array if matrix is not invertible
    
    # TODO: Complete this code
    errors_fwd = np.linalg.norm(H @ points1 - points2, axis=0)
    errors_bwd = np.linalg.norm(H_inv @ points2 - points1, axis=0)

    errors = (errors_fwd + errors_bwd) / 2

    inliers = errors < th

    inliers_indices = np.where(inliers)[0]

    return inliers_indices

Complete the function `Ransac_DLT_homography()`.

In [ ]:
def Ransac_DLT_homography(points1, points2, th, N):
    
    Ncoords, Npts = points1.shape
    
    it = 0
    best_inliers = np.empty(1)
    # TODO: Fill the value of s
    s = 4 # minimum correspondences

    while N > it:
        # Sampling indices and finding inliers
        indices = random.sample(range(Npts), s)
        H = DLT_homography(points1[:,indices], points2[:,indices])
        inliers = find_homography_inliers(H, points1, points2, th)
        
        # test if it is the best model so far
        if len(inliers) > len(best_inliers):
            best_inliers = inliers
        
        it += 1
    
    print(f"Number of iterations: {it}, num inliers: {len(best_inliers)}")
    # compute H from all the inliers
    H = DLT_homography(points1[:,best_inliers], points2[:,best_inliers])
    inliers_recomputed = find_homography_inliers(H, points1, points2, th)
    print(f"Recomputed inliers: {len(inliers_recomputed)}")
    
    return H, best_inliers
    

<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color:rgba(255, 255, 255, 0);">
  <strong>🎥 Video Questions</strong>:
  <ul style="margin-top: 5px; padding-left: 20px;">
    <li>Briefly explain how <b>RANSAC DLT Homography</b> works.</li>
    <li>What is <code>s</code>? Why have you selected that value?</li>
  </ul>
</div>


The following code allows to robustly estimate the homography that relates images 1 and 2. Examine the code and answer the questions below.

In [ ]:
# Homography between images 1 and 2
points1_H12 = []
points2_H12 = []
for m in matches_12:
    points1_H12.append([kp1[m.queryIdx].pt[0], kp1[m.queryIdx].pt[1], 1])
    points2_H12.append([kp2[m.trainIdx].pt[0], kp2[m.trainIdx].pt[1], 1])
    
points1_H12 = np.asarray(points1_H12)
points1_H12 = points1_H12.T
points2_H12 = np.asarray(points2_H12)
points2_H12 = points2_H12.T

# TODO: Find a value for th
th = 1 # max. allowed error in pixels

H_12, indices_inlier_matches_12 = Ransac_DLT_homography(points1_H12, points2_H12, th, 1000)
inlier_matches_12 = [matches_12[i] for i in indices_inlier_matches_12]

img_12 = cv2.drawMatches(img1, kp1, img2, kp2, inlier_matches_12, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plt.figure(figsize=(18.5, 10.5))
plt.title("Inlier correspondences between image 1 and image 2")
plt.imshow(img_12)
plt.show()

<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color:rgba(255, 255, 255, 0);">
  <strong>🎥 Video Questions</strong>:
  <ul style="margin-top: 5px; padding-left: 20px;">
    <li>What is <code>th</code>? Justify the chosen value.</li>
  </ul>
</div>


**Q3.5** Create a new function `Ransac_DLT_homography_adaptive_loop` which is based on the function `Ransac_DLT_homography` and automatically adapts the number of trials to ensure we pick, with a probability $p=0.99$ an initial data set with no outliers.

In [ ]:
def Ransac_DLT_homography_adaptative_loop(points1, points2, th, N):
# TODO: Fill this function
...

    return H, best_inliers
    


**Q3.6** Compare experimentally the two versions of the RANSAC algorithm (`Ransac_DLT_homography` and `Ransac_DLT_homography_adaptive_loop`) in terms of the number of iterations.

In [ ]:
print("Ransac_DLT_homography")
H_12, indices_inlier_matches_12 = Ransac_DLT_homography(points1_H12, points2_H12, th, 1000)

print("\n\nRansac_DLT_homography_adaptative_loop")
H_12, indices_inlier_matches_12 = Ransac_DLT_homography_adaptative_loop(points1_H12, points2_H12, th, 1000)

Now, let's compare how long do these functions take.

In [ ]:
# Suppress all output
with open(os.devnull, 'w') as fnull:
    with contextlib.redirect_stdout(fnull), contextlib.redirect_stderr(fnull):
        time_fixed_loop = %timeit -r 3 -q -o H_12, indices_inlier_matches_12 = Ransac_DLT_homography(points1_H12, points2_H12, th, 1000)
        time_adapt_loop = %timeit -r 3 -q -o H_12, indices_inlier_matches_12 = Ransac_DLT_homography_adaptative_loop(points1_H12, points2_H12, th, 1000)
print(f"Ransac_DLT_homography \t\t\t{time_fixed_loop}")
print(f"Ransac_DLT_homography_adaptative_loop  \t{time_adapt_loop}")

<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color: #e6f2ff;">
  <strong>🎥 Video Questions</strong>:
  <ul style="margin-top: 5px; padding-left: 20px;">
    <li>Given the comparisons of above, which algorithm do you prefer? Briefly explain the difference between the algorithms.</li>
  </ul>
</div>



**Q3.7** Compute the homography that relates images 2 and 3.

In [ ]:
points2_H23 = []
points3_H23 = []

# TODO: Find the homography between images 2 and 3
...


# Plot inlier correspondences
plt.figure(figsize=(18.5, 10.5))
plt.title("Inlier correspondences between image 2 and image 3")
plt.imshow(img_23)
plt.show()

### 3.3 Build the mosaic

At this point we have all the ingredients to build the image mosaic. For transforming an image with a specified homography we use a modification of the function `apply_H` used at the beginning of this session. The modified function `apply_H_fixed_image_size` transforms the input image according to the input homography and writes it in an output image of size corresponding to the input vector of desired corner coordinates.

Examine and complete the code below when necessary.

In [ ]:
# Mosaic corners
MOSAIC_CORNERS = [-400, 1200, -100, 650]

In [ ]:
# Building the mosaic for images 1-2
img1c_w = apply_H_fixed_image_size(img1c, H_12, MOSAIC_CORNERS)
img2c_w = apply_H_fixed_image_size(img2c, np.identity(3), MOSAIC_CORNERS)
img_mosaic_12 = np.maximum(img1c_w, img2c_w)
plt.figure(figsize=(18.5, 10.5))
plt.title("Mosaic with images 1 and 2")
plt.imshow(img_mosaic_12)
plt.show()

<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color: #e6f2ff;">
  <strong>🎥 Video Questions</strong>: Regarding the mosaic with images 1 & 2:
  <ul style="margin-top: 5px; padding-left: 20px;">
    <li>Which homography has been applied to image 1? And to image 2? Why?</li>
  </ul>
</div>


In [ ]:
# TODO: Build the mosaic for images 2-3


In [ ]:
# TODO: Build the mosaic for images 1-2-3


<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color: #e6f2ff;">
  <strong>🎥 Video Questions</strong>: Regarding the mosaic with images 1, 2 & 3:
  <ul style="margin-top: 5px; padding-left: 20px;">
    <li>Does it look right to you?</li>
    <li>Which homography is applied to image 1?</li>
    <li>Which homography is applied to image 2?</li>
    <li>Which homography do you apply to image 3? Why?</li>
  </ul>
</div>


### 3.4 Build your own mosaic **(OPTIONAL)**

Build a mosaic with your own set of images and explain why it works well or it doesn't.

## 4. References

Add here the material you used to complete this Lab. Cite and describe the usage of AI tools if any was used according to the Guidelines for AI tools.

TODO: Complete

<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color: #e6f2ff;">
  <strong>🎥 Video Questions</strong>: Briefly mention the references.
</div>


<div style="border: 2px solid #007acc; border-radius: 10px; padding: 10px; background-color: #e6f2ff;">
  <strong>🎥 Self-Assessment and Conclusions</strong>:
  <ul>
  <li><b>Which parts of the notebook did you succeed in? </b><br>
  <em>Describe the sections where you felt confident, and explain why you think they were successful.</em></li>
  <li><b>Which parts of the notebook did you fail to solve? </b><br>
  <em>Be honest about the areas where you faced difficulties. What challenges or issues did you encounter that you couldn’t resolve? How would you approach these issues in the future?</em></li>
  </ul>
  Is there anything else that you would like to comment?
</div>
